Pylbm for solids.

Pylbm is a python-package for numerical simulations using the Lattice-Boltzmann-Method (LBM). For more general information, refer to the website https://pylbm.readthedocs.io/en/latest/, this documentation will focus on the scope of the project, which is to extend and use pylbm to solve equations of quasi-static linear elasticity in 2D. 

In this document we will learn how to implement source terms.

Source terms are specified in the scheme. Pylbm allows floats or ints for constant source terms or sympy expressions for space and time dependent source terms. For the latter, pylbm reserves the symbols `'X'`, `'Y'`, `'Z'` and `'t'` for symbolic representation of space and time (though it's possible to override this if we provide our own symbols with `'space_x'`, `'space_y'`, `'space_z'` and `'time'` inside `'parameters'`). We can then choose on which of the conserved moments our expressions act. The expressions can include other symbols as long as they are included in `'parameters'`. Keep in mind that half of the forcing is applied before the relaxation and the other half after it, so in the example below the sequence is:
        
        BC
        TRANSPORT
        F2M
        
        m[0] = m[0]+1/2*g_x
        m[1] = m[1]+1/2*g_y
        
        RELAXATION
        
        m[0] = m[0]+1/2*g_x
        m[1] = m[1]+1/2*g_y
        
        M2F
        
Another point to keep in mind is that pylbm will substitute the domain coordinates for X,Y,Z. If we convert to lattice units, our source terms should also be expressed in terms of lattice units     

In [4]:
import sympy as sp
import pylbm

#define coordinate symbols (we have to use exactly these so pylbm knows we are referring to coordinates)
X,Y,t = sp.symbols('X,Y,t')

#define some parameter
C = sp.symbols('C')
c = .5

#define some source terms
g_x = C*(X**2+sp.sin(Y))
g_y = C*(Y*sp.cos(X))

#geometry
dx = 0.1
dt = 0.1
la = dx/dt
L_box = 1.

#characteristic dimensions
U = 1.
L = dx
T = dt

#geometry in lattice units
dx_nd = dx/L
L_box_nd = L_box/L
la_nd = 1.

#material parameters (actual values and in lattice units)
E = 0.1/0.1731
E_nd = T/(L*L)*E
nu = .7
K = E/(2*(1-nu))
K_nd = K*T/L**2
mu = E/(2*(1+nu))
mu_nd = mu*T/L**2
theta = 1/3

#define symbols
u, v, LA = sp.symbols('u, v, LA')
THETA, MU_ND, K_ND, GAMMA = sp.symbols('THETA, MU_ND, K_ND, GAMMA')

#moment matrix
M = sp.Matrix([[1,0,-1,0,1,-1,-1,1],[0,1,0,-1,1,1,-1,-1],[0,0,0,0,1,-1,1,-1],[1,1,1,1,2,2,2,2],
    [1,-1,1,-1,0,0,0,0],[0,0,0,0,1,-1,-1,1],[0,0,0,0,1,1,-1,-1],[GAMMA,GAMMA,GAMMA,GAMMA,1+2*GAMMA,1+2*GAMMA,1+2*GAMMA,1+2*GAMMA]])

#equilibrium moments in order
Meq = [u,v,0,0,0,THETA*u,THETA*v,0]

#relaxation parameters
w10 = 0.
w01 = 0.
w11 = 1/(MU_ND/THETA+.5)
ws = 1/(2*K_ND/(1+THETA)+.5)
wd = 1/(2*MU_ND/(1-THETA)+.5)
w12 = 1.5
w21 = 1.5
wf = 1.
omega = [w10,w01,w11,ws,wd,w12,w21,wf]
gamma = theta*.5/((1+theta)*((1/ws-.5).evalf(subs={K_ND:K_nd,THETA:theta})-.5))

#initial condition
def u_init(x,y):
    return 0.

def v_init(x,y):
    return 0.

dico = {
    'box' : {'x': [0,L_box_nd], 'y': [0,L_box_nd], 'label': -1},
    'space_step': dx_nd,
    'scheme_velocity': LA,
    'parameters': {LA: la_nd,
            THETA: theta,
            MU_ND: mu_nd,
            K_ND: K_nd,
            GAMMA: gamma,
            },
    'init': {u: u_init, v: v_init},
    'generator': 'cython',
    'lbm_algorithm': {'name': pylbm.algorithm.BaseAlgorithm},
    'schemes':[
            {
                'velocities': list(range(1,9)),
                'conserved_moments': [u,v],
                'M': M,
                'equilibrium': Meq,
                'relaxation_parameters': omega,
                'source_terms': {u: g_x, v: g_y}
                }]
    }